In [1]:
import numpy as np
from copy import deepcopy
from typing import Tuple

# Define the transformation functions
def get_preprocess_shape(oldh: int, oldw: int, long_side_length: int) -> Tuple[int, int]:
    """
    Compute the output size given input size and target long side length.
    """
    scale = long_side_length * 1.0 / max(oldh, oldw)
    newh, neww = oldh * scale, oldw * scale
    neww = int(neww + 0.5)
    newh = int(newh + 0.5)
    return (newh, neww)

def inverse_coords(coords: np.ndarray, original_size: Tuple[int, int], target_length=1024) -> np.ndarray:
    """
    Inverse transformation of coordinates from resized back to original.
    """
    old_h, old_w = original_size
    new_h, new_w = get_preprocess_shape(old_h, old_w, target_length)
    coords = deepcopy(coords).astype(float)
    coords[..., 0] = coords[..., 0] * (old_w / new_w)
    coords[..., 1] = coords[..., 1] * (old_h / new_h)
    return coords

def inverse_boxes(boxes: np.ndarray, original_size: Tuple[int, int], target_length=1024) -> np.ndarray:
    """
    Inverse transformation of boxes from resized back to original.
    """
    boxes = inverse_coords(boxes.reshape(-1, 2, 2), original_size, target_length)
    return boxes.reshape(-1, 4)

def convert_boxes(boxes):
    # Convert the boxes from center format to [y_min, x_min, y_max, x_max] format
    converted_boxes = np.zeros_like(boxes)
    converted_boxes[:, 0] = boxes[:, 1] - boxes[:, 3] / 2.0  # x_min
    converted_boxes[:, 1] = boxes[:, 0] - boxes[:, 2] / 2.0  # y_min
    converted_boxes[:, 2] = boxes[:, 1] + boxes[:, 3] / 2.0  # x_max
    converted_boxes[:, 3] = boxes[:, 0] + boxes[:, 2] / 2.0  # y_max
    return converted_boxes


# Load SAM model and generate image embdding

In [2]:
cd /home/icb/hanyi.zhang/Detection_Head/segment-anything

/ictstr01/home/icb/hanyi.zhang/Detection_Head/segment-anything


/home/icb/hanyi.zhang/.local/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
from segment_anything import sam_model_registry, SamPredictor
from segment_anything.separate_sam_encoder import SamEncoder
import torch

sam_checkpoint = "/home/icb/hanyi.zhang/Detection_Head/segment-anything/sam_vit_l_0b3195.pth"
model_type = "vit_l"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

sam_image_encoder = SamEncoder(sam, device)
predictor = SamPredictor(sam)


# Load the Model Checkpoint and evaluate

In [4]:
cd /home/icb/hanyi.zhang/Detection_Head

/ictstr01/home/icb/hanyi.zhang/Detection_Head


/home/icb/hanyi.zhang/.local/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [5]:
# Load checkpoint here
import torch
from detection_head_model import DetectionHead

#checkpoint_path = '/home/icb/hanyi.zhang/Detection_Head/checkpoints/DetectionHead_training-original-epoch=299-val_loss=7.69.ckpt' 
#checkpoint_path = '/home/icb/hanyi.zhang/Detection_Head/checkpoints/DetectionHead_training-smaller-patches-epoch=299-val_loss=9.22.ckpt'
checkpoint_path = '/home/icb/hanyi.zhang/Detection_Head/checkpoints/DetectionHead_training-aug-epoch=299-val_loss=6.30.ckpt'

model = DetectionHead.load_from_checkpoint(checkpoint_path)

# Set the model to evaluation mode
model.eval()

# Determine device (GPU or CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

DetectionHead(
  (matcher): HungarianMatcher()
  (criterion): SetCriterion(
    (matcher): HungarianMatcher()
  )
  (query_embed): Embedding(100, 256)
  (position_embedding): PositionEmbeddingSine()
  (transformer_decoder): TransformerDecoder(
    (self_attention_layers): ModuleList(
      (0-5): 6 x SelfAttentionLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
        )
        (norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (cross_attention_layers): ModuleList(
      (0-5): 6 x CrossAttentionLayer(
        (multihead_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
        )
        (norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (ffn_layers): ModuleL

In [6]:
import os
import numpy as np
from PIL import Image
import cv2

# Function to load checkpoint and perform evaluation on one example
def evaluate_single_example(model, img_emb):
    # Prepare the input
    img_emb = torch.tensor(img_emb, dtype=torch.float32).to(device)  # Convert and move to GPU if available
    
    # Dummy query embedding (you may adjust this based on your actual implementation)
    query_embed = model.query_embed.weight.to(device)
    
    # Get positional embedding
    pos_embedding = model.position_embedding(img_emb)
    
    # Forward pass
    with torch.no_grad():
        outputs = model(query_embedding=query_embed, image_embedding=img_emb, pos_embedding=pos_embedding)

    # Apply softmax to pred_logits
    pred_logits = torch.nn.functional.softmax(outputs['pred_logits'][0], dim=-1)
    
    # Convert softmax output to binary predictions (0 or 1)
    binary_predictions = (pred_logits[:, 0] > pred_logits[:, 1]).cpu().numpy().astype(int)
    
    # Filter out boxes with binary prediction value 0
    pred_boxes = outputs['pred_boxes'].cpu().numpy()[0]
    filtered_boxes = pred_boxes[binary_predictions == 1]
    confidence_scores = pred_logits[binary_predictions == 1, 0].cpu().numpy()
    
    return filtered_boxes, confidence_scores

def non_max_suppression(bboxes, scores, threshold):
    """Perform Non-Maximum Suppression (NMS) on bounding boxes.
    
    Args:
    bboxes (numpy.ndarray): Array of bounding boxes in the format (x1, y1, x2, y2).
    scores (numpy.ndarray): Array of scores for each bounding box.
    threshold (float): IoU threshold for suppression.

    Returns:
    numpy.ndarray: Array of indices of bounding boxes to keep.
    """
    # Sort the bounding boxes by the scores in descending order
    indices = np.argsort(scores)[::-1]
    
    keep = []
    while len(indices) > 0:
        current = indices[0]
        keep.append(current)
        
        if len(indices) == 1:
            break
        
        current_box = bboxes[current]
        remaining_boxes = bboxes[indices[1:]]
        
        # Compute IoU of the current box with the rest
        ious = np.array([compute_iou(current_box, box) for box in remaining_boxes])
        
        # Select boxes with IoU less than the threshold
        indices = indices[1:][ious < threshold]
    
    return np.array(keep)

def compute_iou(box1, box2):
    x1_max = max(box1[0], box2[0])
    y1_max = max(box1[1], box2[1])
    x2_min = min(box1[2], box2[2])
    y2_min = min(box1[3], box2[3])

    intersection_area = max(0, x2_min - x1_max) * max(0, y2_min - y1_max)

    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])

    union_area = box1_area + box2_area - intersection_area

    if union_area == 0:
        return 0.0

    iou = intersection_area / union_area
    return iou



# Images from Neurips Dataset

In [7]:
import os
import numpy as np
import cv2
import torch
from PIL import Image

def transform_mask(mask):
    """
    Transforms a mask of shape (N, 1, H, W) into a single mask of shape (H, W),
    where background is 0 and different masks are labeled as 1, 2, 3, ..., N.
    If the mask contains only zeros, it returns a zero-filled mask of shape (H, W).
    """
    # Check if the mask is entirely zeros
    if len(mask.shape) == 2:
        return mask

    # Get the number of masks
    num_masks = mask.shape[0]
    
    # Initialize a new mask with zeros (background)
    transformed_mask = np.zeros((mask.shape[2], mask.shape[3]), dtype=np.uint8)

    # Assign label 1, 2, 3, ..., N to each mask respectively
    for i in range(num_masks):
        single_mask = mask[i, 0, :, :]  # Extract each mask
        transformed_mask[single_mask > 0] = i + 1  # Label the mask with 1, 2, ..., N

    return transformed_mask

# Define paths
base_folder = '/home/icb/hanyi.zhang/NeurIPS22-CellSeg-png-aug/test'
img_folder = os.path.join(base_folder, 'images_patches')
img_emb_folder = os.path.join(base_folder, 'images_patches_emb')
predictions_save_path = '/home/icb/hanyi.zhang/cellseg_evaluation_results/ours_masks_test_th05'
bboxes_save_path = '/home/icb/hanyi.zhang/cellseg_evaluation_results/ours_bboxes_test_th05'
os.makedirs(predictions_save_path, exist_ok=True)
os.makedirs(bboxes_save_path, exist_ok=True)

# Define NMS threshold
threshold = 0.5

# Get list of files
img_emb_files = os.listdir(img_emb_folder)
i = 1
for img_emb_file in img_emb_files:
    file_idx = img_emb_file.rsplit('.', 1)[0]
    #print(file_idx)

    # Define image path, label path and image embedding path
    image_path = os.path.join(img_folder, file_idx + '.png')
    image_embedding_path = os.path.join(img_emb_folder, img_emb_file)
    mask_save_path = os.path.join(predictions_save_path, file_idx + '.npy')
    bbox_save_path = os.path.join(bboxes_save_path, file_idx + '_bboxes.npy')

    # Read the image
    image = Image.open(image_path)

    # Get the original size of the image
    original_size = image.size  # (width, height)
    original_size = (original_size[1], original_size[0])  # Convert to (height, width)

    # Load image embedding
    image_embedding = np.load(image_embedding_path)

    # Predict boxes and convert to original size
    predicted_boxes, confidence_scores = evaluate_single_example(model, image_embedding)
    original_boxes_pred = inverse_boxes(convert_boxes(predicted_boxes) * 1024, original_size)

    kept_indices = non_max_suppression(original_boxes_pred, confidence_scores, threshold)
    if len(kept_indices) != 0:
        original_boxes_pred_filtered = original_boxes_pred[kept_indices]
        filtered_confidences = confidence_scores[kept_indices]
        
        # Predict masks
        image = cv2.imread(image_path)
        predictor.set_image(image)
        input_boxes = torch.tensor(original_boxes_pred_filtered).to(device)
        transformed_boxes = predictor.transform.apply_boxes_torch(input_boxes, image.shape[:2])
        masks, _, _ = predictor.predict_torch(
            point_coords=None,
            point_labels=None,
            boxes=transformed_boxes,
            multimask_output=False,
        )
        predicted_masks = masks.cpu().numpy()
        predicted_masks = transform_mask(predicted_masks)

        # Save the bounding boxes
        np.save(bbox_save_path, original_boxes_pred_filtered)
    else:
        # If no boxes, create an all-zero mask and save empty bounding boxes
        predicted_masks = np.zeros((original_size[0], original_size[1]), dtype=np.uint8)
        np.save(bbox_save_path, np.array([]))  # Save empty array for bounding boxes

    # Save the mask
    np.save(mask_save_path, predicted_masks)

    if i % 100 == 0:
        print(f"Processed {i} images")
    i += 1


cell_00050_img_patch_20
cell_00054_img_patch_8
cell_00043_img_patch_1
cell_00077_img_patch_5
cell_00087_img_patch_1
cell_00031_img_patch_5
cell_00077_img_patch_25
cell_00061_img_patch_30
cell_00034_img_patch_0
cell_00053_img_patch_12
cell_00098_img_patch_0
cell_00057_img_patch_7
cell_00056_img_patch_8
cell_00054_img_patch_29
cell_00078_img_patch_7
cell_00071_img_patch_5
cell_00050_img_patch_13
cell_00100_img_patch_8
cell_00064_img_patch_5
cell_00049_img_patch_0
cell_00064_img_patch_8
cell_00038_img_patch_5
cell_00072_img_patch_2
cell_00041_img_patch_12
cell_00062_img_patch_32
cell_00053_img_patch_5
cell_00059_img_patch_5
cell_00099_img_patch_32
cell_00065_img_patch_4
cell_00100_img_patch_4
cell_00092_img_patch_3
cell_00058_img_patch_13
cell_00075_img_patch_9
cell_00085_img_patch_2
cell_00061_img_patch_33
cell_00080_img_patch_2
cell_00061_img_patch_35
cell_00048_img_patch_10
cell_00041_img_patch_11
cell_00045_img_patch_3
cell_00054_img_patch_14
cell_00064_img_patch_13
cell_00071_img_pat

# New test
### Generate masks with GT boxes

In [8]:
'''import os
import numpy as np
import cv2
import torch
from PIL import Image

# Define paths
base_folder = '/home/icb/hanyi.zhang/NeurIPS22-CellSeg-png/test'
img_folder = os.path.join(base_folder, 'images_patches')
img_emb_folder = os.path.join(base_folder, 'images_patches_emb')
predictions_save_path = '/home/icb/hanyi.zhang/evaluation_results/ours_masks_in_GT_boxes'
gt_boxes_path = '/home/icb/hanyi.zhang/NeurIPS22-CellSeg-png/test/targets'
os.makedirs(predictions_save_path, exist_ok=True)

# Get list of files
img_emb_files = os.listdir(img_emb_folder)
i = 1
for img_emb_file in img_emb_files:
    file_idx = img_emb_file.rsplit('.', 1)[0]
    #print(file_idx)

    # Define image path, label path and image embedding path
    image_path = os.path.join(img_folder, file_idx + '.png')
    image_embedding_path = os.path.join(img_emb_folder, img_emb_file)
    mask_save_path = os.path.join(predictions_save_path, file_idx + '.npy')
    bbox_save_path = os.path.join(bboxes_save_path, file_idx + '_bboxes.npy')

    # Read the image
    image = Image.open(image_path)

    # Get the original size of the image
    original_size = image.size  # (width, height)
    original_size = (original_size[1], original_size[0])  # Convert to (height, width)

    # Load image embedding
    image_embedding = np.load(image_embedding_path)

    # Predict boxes and convert to original size
    #predicted_boxes, confidence_scores = evaluate_single_example(model, image_embedding)
    # use GT boxes instead
    predicted_boxes = np.load(os.path.join(gt_boxes_path, file_idx + '.npy'))

    original_boxes_pred = inverse_boxes(convert_boxes(predicted_boxes) * 1024, original_size)

    # Predict masks
    image = cv2.imread(image_path)
    predictor.set_image(image)
    input_boxes = torch.tensor(original_boxes_pred).to(device)
    transformed_boxes = predictor.transform.apply_boxes_torch(input_boxes, image.shape[:2])
    masks, _, _ = predictor.predict_torch(
        point_coords=None,
        point_labels=None,
        boxes=transformed_boxes,
        multimask_output=False,
    )
    predicted_masks = masks.cpu().numpy()

    # Save the mask
    np.save(mask_save_path, predicted_masks)

    if i % 100 == 0:
        print(f"Processed {i} images")
    i += 1

'''

'import os\nimport numpy as np\nimport cv2\nimport torch\nfrom PIL import Image\n\n# Define paths\nbase_folder = \'/home/icb/hanyi.zhang/NeurIPS22-CellSeg-png/test\'\nimg_folder = os.path.join(base_folder, \'images_patches\')\nimg_emb_folder = os.path.join(base_folder, \'images_patches_emb\')\npredictions_save_path = \'/home/icb/hanyi.zhang/evaluation_results/ours_masks_in_GT_boxes\'\ngt_boxes_path = \'/home/icb/hanyi.zhang/NeurIPS22-CellSeg-png/test/targets\'\nos.makedirs(predictions_save_path, exist_ok=True)\n\n# Get list of files\nimg_emb_files = os.listdir(img_emb_folder)\ni = 1\nfor img_emb_file in img_emb_files:\n    file_idx = img_emb_file.rsplit(\'.\', 1)[0]\n    #print(file_idx)\n\n    # Define image path, label path and image embedding path\n    image_path = os.path.join(img_folder, file_idx + \'.png\')\n    image_embedding_path = os.path.join(img_emb_folder, img_emb_file)\n    mask_save_path = os.path.join(predictions_save_path, file_idx + \'.npy\')\n    bbox_save_path = os.